## Import the Modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [3]:
pd.set_option('display.max_columns', None)
plt.style.use('default')

print("Setup complete")

Setup complete


## Load the data

In [4]:
train = pd.read_csv("C:\\Users\\kalah\\Downloads\\Train.csv")
test = pd.read_csv("C:\\Users\\kalah\\Downloads\\Test.csv")
sample_sub = pd.read_csv("C:\\Users\\kalah\\Downloads\\SampleSubmission.csv")
var_defs = pd.read_csv("C:\\Users\\kalah\\Downloads\\VariableDefinitions.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)


Train shape: (9618, 39)
Test shape: (2405, 38)


## Look at Data

In [5]:
train.head()

,ID,country,owner_age,attitude_stable_business_environment,attitude_worried_shutdown,compliance_income_tax,perception_insurance_doesnt_cover_losses,perception_cannot_afford_insurance,personal_income,business_expenses,business_turnover,business_age_years,motor_vehicle_insurance,has_mobile_money,current_problem_cash_flow,has_cellphone,owner_sex,offers_credit_to_customers,attitude_satisfied_with_achievement,has_credit_card,keeps_financial_records,perception_insurance_companies_dont_insure_businesses_like_yours,perception_insurance_important,has_insurance,covid_essential_service,attitude_more_successful_next_year,problem_sourcing_money,marketing_word_of_mouth,has_loan_account,has_internet_banking,has_debit_card,future_risk_theft_stock,business_age_months,medical_insurance,funeral_insurance,motivation_make_more_money,uses_friends_family_savings,uses_informal_lender,Target
0,ID_3CFL0U,eswatini,63.0,Yes,No,No,No,Yes,3000.0,6000.0,7000.0,14.0,Never had,Have now,Yes,Yes,Male,"Yes, sometimes",No,Never had,"Yes, always",Yes,Yes,No,Yes,Yes,Yes,Yes,Never had,Never had,Never had,NaN,6.0,Never had,Used to have but don’t have now,NaN,Never had,Never had,Low
1,ID_XWI7G3,zimbabwe,39.0,No,Yes,Yes,No,Yes,NaN,NaN,NaN,15.0,Have now,Have now,NaN,Yes,Male,"Yes, sometimes",Yes,Never had,"Yes, always",No,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,No,3.0,Never had,Never had,NaN,NaN,NaN,Medium
2,ID_TY93LV,malawi,34.0,Don’t know or N/A,No,No,Don't know,Yes,30000.0,6000.0,13000.0,5.0,NaN,Never had,Yes,Yes,Male,"Yes, sometimes",Yes,Never had,No,Don?t know / doesn?t apply,Yes,No,NaN,Yes,Yes,No,Never had,Never had,Never had,Yes,NaN,NaN,NaN,Yes,NaN,NaN,Low
3,ID_9OP2C8,malawi,28.0,Yes,No,No,No,No,180000.0,60000.0,30000.0,1.0,NaN,Have now,No,Yes,Female,"Yes, sometimes",No,Never had,No,No,Yes,No,NaN,Yes,No,No,Never had,Never had,Never had,No,NaN,NaN,NaN,Yes,Never had,Have now,Low
4,ID_13REYS,zimbabwe,43.0,Yes,No,No,Yes,Yes,50.0,2400.0,1800.0,3.0,Never had,NaN,NaN,No,Female,"Yes, sometimes",Yes,Never had,No,Yes,Yes,No,No,NaN,NaN,NaN,NaN,NaN,NaN,No,0.0,Never had,Never had,Yes,NaN,NaN,Low


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9618 entries, 0 to 9617
Data columns (total 39 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   ID                                                                9618 non-null   object 
 1   country                                                           9618 non-null   object 
 2   owner_age                                                         9618 non-null   float64
 3   attitude_stable_business_environment                              9616 non-null   object 
 4   attitude_worried_shutdown                                         9616 non-null   object 
 5   compliance_income_tax                                             9614 non-null   object 
 6   perception_insurance_doesnt_cover_losses                          9613 non-null   object 
 7   perception_cannot_afford_insuranc

In [7]:
target_col = sample_sub.columns[1]
print("Target column:", target_col)

train[target_col].value_counts()


Target column: Target


Target
Low       6280
Medium    2868
High       470
Name: count, dtype: int64

In [8]:
train[target_col].value_counts(normalize=True)


Target
Low       0.652942
Medium    0.298191
High      0.048867
Name: proportion, dtype: float64

## Separate Features & Target

In [9]:
X = train.drop(columns=[target_col])
y = train[target_col]

test_X = test.copy()

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (9618, 38)
y shape: (9618,)


## Identify Numerical and Categorical Columns

In [10]:
num_cols = X.select_dtypes(include='number').columns
cat_cols = X.select_dtypes(exclude='number').columns

print("Numerical columns:", len(num_cols))
print("Categorical columns:", len(cat_cols))


Numerical columns: 6
Categorical columns: 32


## Handle Missing Values

In [11]:
X.isnull().sum()

ID                                                                     0
country                                                                0
owner_age                                                              0
attitude_stable_business_environment                                   2
attitude_worried_shutdown                                              2
compliance_income_tax                                                  4
perception_insurance_doesnt_cover_losses                               5
perception_cannot_afford_insurance                                     5
personal_income                                                      109
business_expenses                                                    229
business_turnover                                                    216
business_age_years                                                   252
motor_vehicle_insurance                                             2244
has_mobile_money                                   

In [12]:
X[num_cols] = X[num_cols].fillna(X[num_cols].median())
test_X[num_cols] = test_X[num_cols].fillna(X[num_cols].median())


In [13]:
X.isnull().sum()

ID                                                                     0
country                                                                0
owner_age                                                              0
attitude_stable_business_environment                                   2
attitude_worried_shutdown                                              2
compliance_income_tax                                                  4
perception_insurance_doesnt_cover_losses                               5
perception_cannot_afford_insurance                                     5
personal_income                                                        0
business_expenses                                                      0
business_turnover                                                      0
business_age_years                                                     0
motor_vehicle_insurance                                             2244
has_mobile_money                                   

In [14]:
for col in cat_cols:
    mode_val = X[col].mode()[0]
    X[col] = X[col].fillna(mode_val)
    test_X[col] = test_X[col].fillna(mode_val)


In [15]:
X.isnull().sum()

ID                                                                  0
country                                                             0
owner_age                                                           0
attitude_stable_business_environment                                0
attitude_worried_shutdown                                           0
compliance_income_tax                                               0
perception_insurance_doesnt_cover_losses                            0
perception_cannot_afford_insurance                                  0
personal_income                                                     0
business_expenses                                                   0
business_turnover                                                   0
business_age_years                                                  0
motor_vehicle_insurance                                             0
has_mobile_money                                                    0
current_problem_cash

## Encode categorical Features

In [16]:
from sklearn.preprocessing import LabelEncoder

target_encoder = LabelEncoder()

y_encoded = target_encoder.fit_transform(y)


In [17]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

X[cat_cols] = encoder.fit_transform(X[cat_cols])
test_X[cat_cols] = encoder.transform(test_X[cat_cols])


## Train Validation Split

In [18]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded,
    test_size=0.2,
    stratify=y_encoded,
    random_state=42
)


print("Train size:", X_train.shape)
print("Validation size:", X_val.shape)


Train size: (7694, 38)
Validation size: (1924, 38)


## Baseline Model

In [19]:
# Build class_weight dynamically from encoder
class_weight = {
    target_encoder.transform(["Low"])[0]: 1,
    target_encoder.transform(["Medium"])[0]: 2,
    target_encoder.transform(["High"])[0]: 4,
}

print(class_weight)

seeds = [11, 22, 33, 44, 55, 66, 77, 88, 99, 111]

all_test_probs = []

for seed in seeds:
    print(f"Training RF with seed {seed}")

    rf = RandomForestClassifier(
        n_estimators=600,
        max_depth=None,
        min_samples_leaf=2,
        class_weight=class_weight,   # the CORRECT mapping
        random_state=seed,
        n_jobs=-1
    )

    rf.fit(X, y_encoded)
    all_test_probs.append(rf.predict_proba(test_X))




{np.int64(1): 1, np.int64(2): 2, np.int64(0): 4}
Training RF with seed 11
Training RF with seed 22
Training RF with seed 33
Training RF with seed 44
Training RF with seed 55
Training RF with seed 66
Training RF with seed 77
Training RF with seed 88
Training RF with seed 99
Training RF with seed 111


In [20]:
avg_test_probs = np.mean(all_test_probs, axis=0)

final_preds_encoded = avg_test_probs.argmax(axis=1)
final_preds = target_encoder.inverse_transform(final_preds_encoded)



## Evaluate the Model

In [25]:
# val_preds = rf.predict(X_val)

# print(classification_report(y_val, val_preds))


## Train on full data

In [22]:
rf.fit(X, y_encoded)



,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",600
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",2
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

## Predict Test

In [23]:


# RF probabilities only
# rf.fit(X, y_encoded)

# # # Plain prediction
# final_preds_encoded = rf.predict(test_X)

# # # Decode
# final_preds = target_encoder.inverse_transform(final_preds_encoded)
       
submission = sample_sub.copy()
submission[target_col] = final_preds
submission.to_csv("submissionLast.csv", index=False)

# submission = sample_sub.copy()
# submission[target_col] = final_preds



In [24]:

# submission.to_csv("submission_final2.csv", index=False)

# print("submission_final2.csv created ")

